In [1]:
import itertools

In [2]:
import cobra
import cobra.test
import cobra.io
import pathlib
import pickle

In [3]:
data_dir = cobra.test.data_dir
data_path = pathlib.Path(data_dir)
file_path = data_path / "iJO1366.pickle"
with file_path.open('rb') as f:
    model = pickle.load(f)

In [4]:
cobra.io.write_sbml_model(model, "../models/iJO1366.xml")

In [5]:
!head models/iJO1366.xml

<?xml version="1.0" encoding="UTF-8"?>
<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" xmlns:fbc="http://www.sbml.org/sbml/level3/version1/fbc/version2" xmlns:groups="http://www.sbml.org/sbml/level3/version1/groups/version1" metaid="meta_" sboTerm="SBO:0000624" level="3" version="1" fbc:required="false" groups:required="false">
  <model metaid="meta_iJO1366" id="iJO1366" name="Escherichia coli str. K-12 substr. MG1655" fbc:strict="true">
    <notes>
      <html xmlns="http://www.w3.org/1999/xhtml">
        <p>This is a metabolism model of Escherichia coli str. K-12 substr. MG1655 in 
        <a href="http: //sbml.org" target="_blank" title="Access the definition of the SBML file format.">SBML</a> format.</p>
        <p>Redistribution and use of any part of this model from BiGG Models knowledge-base, with or without modification, are permitted provided that the following conditions are met: <ol>
          <li>Redistributions of this SBML file must retain the above copyright 

In [6]:
import json

In [7]:
!curl -O 'http://bigg.ucsd.edu/static/models/iJO1366.json'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2879k  100 2879k    0     0  2876k      0  0:00:01  0:00:01 --:--:-- 2876k


In [8]:
with open('./iJO1366.json') as f:
    _info = json.load(f)
_info

info = {}
for entry in itertools.chain(_info['metabolites'], _info['reactions']):
    assert entry['id'] not in info
    entry = entry.copy()
    if 'compartment' in entry:
        entry['compartment'] = _info['compartments'][entry['compartment']]
    info[entry['id']] = entry
info.update(_info['compartments'])
info

{'10fthf_c': {'id': '10fthf_c',
  'name': '10-Formyltetrahydrofolate',
  'compartment': 'cytosol',
  'charge': -2,
  'formula': 'C20H21N7O7',
  'notes': {'original_bigg_ids': ['10fthf_c']},
  'annotation': {'bigg.metabolite': ['10fthf'],
   'biocyc': ['META:10-FORMYL-THF'],
   'chebi': ['CHEBI:19108',
    'CHEBI:15637',
    'CHEBI:698',
    'CHEBI:19109',
    'CHEBI:11304',
    'CHEBI:57454'],
   'hmdb': ['HMDB00972'],
   'inchi_key': ['AUFGTPPARQZWDO-YPMHNXCESA-L'],
   'kegg.compound': ['C00234'],
   'metanetx.chemical': ['MNXM237'],
   'reactome.compound': ['5389850', '419151'],
   'sabiork': ['1318'],
   'sbo': 'SBO:0000247',
   'seed.compound': ['cpd00201']}},
 '12dgr120_c': {'id': '12dgr120_c',
  'name': '1,2-Diacyl-sn-glycerol (didodecanoyl, n-C12:0)',
  'compartment': 'cytosol',
  'charge': 0,
  'formula': 'C27H52O5',
  'notes': {'original_bigg_ids': ['12dgr120_c']},
  'annotation': {'bigg.metabolite': ['12dgr120'],
   'metanetx.chemical': ['MNXM4939'],
   'sbo': 'SBO:0000247'}}

In [9]:
with open('./iJO1366Centralmetabolism.cyjs') as f:
    data = json.load(f)
data

{'elements': {'nodes': [{'data': {'id': '2075291',
     'x': 3006.858,
     'y': 3627.7302,
     'node_type': 'metabolite',
     'bigg_id': 'mal__L_c',
     'label': 'L-Malate'},
    'position': {'x': 3006.858, 'y': 3627.7302},
    'selected': 'false'},
   {'data': {'id': '2075292',
     'x': 3048.1480390625,
     'y': 3786.964575,
     'node_type': 'metabolite',
     'bigg_id': 'nad_c',
     'label': 'Nicotinamide adenine dinucleotide'},
    'position': {'x': 3048.1480390625, 'y': 3786.964575},
    'selected': 'false'},
   {'data': {'id': '2075293',
     'x': 3065.1851484375,
     'y': 4046.6335203125,
     'node_type': 'metabolite',
     'bigg_id': 'nadh_c',
     'label': 'Nicotinamide adenine dinucleotide - reduced'},
    'position': {'x': 3065.1851484375, 'y': 4046.6335203125},
    'selected': 'false'},
   {'data': {'id': '2075294',
     'x': 3058.6460859375,
     'y': 3978.652075,
     'node_type': 'metabolite',
     'bigg_id': 'h_c',
     'label': 'H+'},
    'position': {'x': 305

In [10]:
for element in data['elements']['nodes']:
    if element['data']['node_type'] == 'metabolite':
        # element_id = 'M_{}'.format(element['data']['bigg_id'])
        element_id = element['data']['bigg_id']
        element['data']['name'] = element_id
        element['data']['shared_name'] = element_id
        # del element['data']['bigg_id']
        # del element['data']['label']
        if element_id in info:
            entry = info[element_id]
            for key in ('compartment', ):
                if key in entry and entry[key] != '':
                    element['data'][key] = entry[key]
    else:
        # print(element)
        assert element['data']['node_type'] in ('midmarker', 'multimarker')

for element in data['elements']['edges']:
    # element_id = 'R_{}'.format(element['data']['bigg_id'])
    element_id = element['data']['bigg_id']
    element['data']['name'] = element_id
    element['data']['shared_name'] = element_id
    
    if element_id in info:
        entry = info[element_id]
        for key in ('subsystem', ):
            if key in entry and entry[key] != '':
                element['data'][key] = entry[key]

In [11]:
data

{'elements': {'nodes': [{'data': {'id': '2075291',
     'x': 3006.858,
     'y': 3627.7302,
     'node_type': 'metabolite',
     'bigg_id': 'mal__L_c',
     'label': 'L-Malate',
     'name': 'mal__L_c',
     'shared_name': 'mal__L_c',
     'compartment': 'cytosol'},
    'position': {'x': 3006.858, 'y': 3627.7302},
    'selected': 'false'},
   {'data': {'id': '2075292',
     'x': 3048.1480390625,
     'y': 3786.964575,
     'node_type': 'metabolite',
     'bigg_id': 'nad_c',
     'label': 'Nicotinamide adenine dinucleotide',
     'name': 'nad_c',
     'shared_name': 'nad_c',
     'compartment': 'cytosol'},
    'position': {'x': 3048.1480390625, 'y': 3786.964575},
    'selected': 'false'},
   {'data': {'id': '2075293',
     'x': 3065.1851484375,
     'y': 4046.6335203125,
     'node_type': 'metabolite',
     'bigg_id': 'nadh_c',
     'label': 'Nicotinamide adenine dinucleotide - reduced',
     'name': 'nadh_c',
     'shared_name': 'nadh_c',
     'compartment': 'cytosol'},
    'position':

In [12]:
with open('../models/iJO1366.cyjs', 'w') as f:
    json.dump(data, f, indent=2)

In [13]:
!head models/iJO1366.cyjs

{
  "elements": {
    "nodes": [
      {
        "data": {
          "id": "2075291",
          "x": 3006.858,
          "y": 3627.7302,
          "node_type": "metabolite",
          "bigg_id": "mal__L_c",
